In [6]:
import os, csv, subprocess

import numpy as np
import pandas as pd

In [4]:
def find_unique_areas(areas):
    uniques, counts = np.unique(areas, return_counts = True)
    counts = counts.astype(str)
    area_list = []
    count_list = []
    area_list.append(uniques)
    count_list.append(counts)
    return np.vstack((area_list, count_list)).T 

### Voitalov et al. (2019) published a python script to identify power law distributions. This notebook automates the steps needed to check the distributions produced by the model simulations. 

#### Please refer to Voitalov et al. (2019) for more detail on identifying power laws:
#### Voitalov, I., Van Der Hoorn, P., Van Der Hofstad, R., & Krioukov, D. (2019). Scale-free networks well done. Physical Review Research, 1(3). https://doi.org/10.1103/PhysRevResearch.1.033034

#### See https://github.com/ivanvoitalov/tail-estimation for step-by-step instructions on using their tail-estimation script. 

#### The script below combines the output from each simulation for a given set of parameters into a .dat file. It repeats the process for each parameter set to produce 32 .dat files.

In [7]:
d = "model-data/csv-files" #path to directory of simulation CSVs

#Go through each sub-directory
for d1 in os.listdir(d):
    if not d1.startswith('.'):  #ignore hidden files  
        x=os.path.join(d,d1)
        for d2 in os.listdir(x):
            if not d2.startswith('.'):
                y=os.path.join(x,d2)
                for d3 in os.listdir(y):
                    z = os.path.join(y, d3)
                    if not d3.startswith('.'):
                        #Append the area column from the final simulation timestep
                        areasList = []
                        i = 0
                        for f in os.listdir(z):
                            if i == 0: 
                                fullname = os.path.splitext(f)[0][:-2] #take the name from the first simulation file
                            df = pd.read_csv(os.path.join(z, f))
                            last_step = df.iloc[:,-1]
                            areasList.append(last_step.iloc[:])
                            i += 1
                        areasList = np.array(areasList)
                        areasList = areasList[~np.isnan(areasList)]
                        count = find_unique_areas(areasList)

                        simname = '_' + fullname + '_combined'

                        #Write the combined list of areas and counts to a .dat file
                        fname = os.path.join("model-data/combined-dat-files", simname + '.dat')
                        with open(fname, 'w', newline = '') as datfile:
                            writer = csv.writer(datfile, delimiter = ' ')
                            for i in range(len(count)):
                                writer.writerow(count[i,:])

#### Now run "voitalov-tail-estimation.py" on each of the simulation .dat files. Alternatively, you can open a terminal and run the script on individual .dat files. See https://github.com/ivanvoitalov/tail-estimation for more information. 

In [ ]:
#Does not run on mac, check to see if it runs on PC
for f in os.listdir("model-data/combined-dat-files/"):
    if not f.startswith('.'):
        if f.endswith('.dat'):
            n = os.path.splitext(f)[0]
            if os.path.isfile(n + '.pdf') == False:
                subprocess.call(['python', 'voitalov-tail-estimation.py', os.path.join("model-data/combined-dat-files/",f), n + '_new.pdf'])